In [1]:
#Import modules:
import pipistrello
database_dir = ('/home/juan/MHPC-Thesis/NetCDF_files')
all_cubes = pipistrello.database(database_dir)

Cleaning up compiled files...
compiled_krb: No such file or directory.
	 __pycache__/utils.cpython-34.pyc removed
	 __pycache__/pipistrello.cpython-34.pyc removed
	 __pycache__ removed
loading database...
FILE:

0, -1, u.nc

CUBE:

0, 0, Vector wind eastward component

METADATA:

0, 0, CubeMetadata(standard_name=None, long_name='Vector wind eastward component', var_name='u', units=Unit('meter per second'), attributes={'WOA01': 'http://localhost/junk', 'DODS_Name': 'UWind', 'base_time': '88- 10-00:00:00', 'i': 32000, 'b': -128, 'DODS_EXTRA.Unlimited_Dimension': 'time_a', 'title': ' FNOC UV wind components from 1988- 10 to 1988- 13.'}, cell_methods=())

FILE:

1, -1, coords.nc

FILE:

2, -1, new_mslp.nc

CUBE:

2, 0, air_pressure_at_sea_level

COORDINATE:

2, 0, time, 3888000.0, 19440000.0, , seconds since 2009-01-01

COORDINATE:

2, 0, latitude, 22.5, 52.5, , degrees

COORDINATE:

2, 0, longitude, -132.5, -72.5, , degrees

METADATA:

2, 0, CubeMetadata(standard_name='air_pressure_at_sea

In [4]:
#
print(len(all_cubes.datafiles),
      len(all_cubes.cubes),
      len(all_cubes.coordinates),
           len(all_cubes.metadatas))


7 7 7 7


In [14]:
for i in range(len(all_cubes.datafiles)):
    for j in range(len(all_cubes.cubes[i])):
    print(all_cubes.metadatas[i],"\n-----\n")

["CubeMetadata(standard_name=None, long_name='Vector wind eastward component', var_name='u', units=Unit('meter per second'), attributes={'WOA01': 'http://localhost/junk', 'DODS_Name': 'UWind', 'base_time': '88- 10-00:00:00', 'i': 32000, 'b': -128, 'DODS_EXTRA.Unlimited_Dimension': 'time_a', 'title': ' FNOC UV wind components from 1988- 10 to 1988- 13.'}, cell_methods=())"] 
-----

[] 
-----

["CubeMetadata(standard_name='air_pressure_at_sea_level', long_name=None, var_name='pres', units=Unit('hectopascal'), attributes={'title': 'example for workshop'}, cell_methods=())"] 
-----

["CubeMetadata(standard_name='air_pressure_at_sea_level', long_name=None, var_name='pr', units=Unit('hectopascal'), attributes={'title': 'example for workshop'}, cell_methods=())"] 
-----

["CubeMetadata(standard_name=None, long_name='Vector wind northward component', var_name='v', units=Unit('meter per second'), attributes={'DODS_EXTRA.Unlimited_Dimension': 'time_a', 'DODS_Name': 'VWind', 'base_time': '88- 10-

In [ ]:
#tomorrow start from here!
def get_pipistrello_cubes(cube_name=''):
    indices_to_load = []
    cube_name = cube_name.lower()
    for i in range(len(all_cubes.datafiles)):
        for j in range(len(all_cubes.cubes[i])):
            if cube_name in all_cubes.cubes[i][j].lower():
                indices_to_load.append([i,j])

    pipistrello_cubes = []
    for indices in indices_to_load:
        pipistrello_cubes.append(
            iris.load(all_cubes.location+all_cubes.datafiles[indices[0]],
                      all_cubes.cubes[indices[0]][indices[1]])
                       )
    if len(pipistrello_cubes) < 1:
        print('No cubes found with "{}" in its name'.format(cube_name))
        print("Did you mean...?")
        
        for i in range(len(all_cubes.datafiles)):
            for j in range(len(all_cubes.cubes[i])):
                for k in range(0,len(cube_name)-3):
                    if cube_name[k:k+3] in all_cubes.cubes[i][j].lower():
                        print(all_cubes.cubes[i][j])
                        break
                    
    return pipistrello_cubes

In [ ]:
cubesA = all_cubes.load_cubes('thward')
for cube in cubesA:
    print(cube)

In [ ]:
line1 = "asdf\n"
line2 = "asdf"
print(line1[:-1])
print(line2)



In [ ]:
pipistrello.create_database(database_dir)

In [ ]:
print(all_cubes)

In [ ]:
for c in cubes:
    if c.name() == 'AIR TEMPERATURE':
        c0 = c
print(c0)


In [ ]:
for coord in c0.coords():
    print(coord.name())

In [ ]:
print(c0.coord('latitude'))
print(c0.coord('longitude'))

In [ ]:
#This will give error:
#ValueError: 
#zero not allowed as a reference year, does not exist in Julian or Gregorian calendars

if(0): print(c0.coord('TIME'))

#Why? ...See the error and next box for the solution.

In [ ]:
#Reason for the error above: The reference year is set to zero...
#See...
print(c0.coord('TIME').units)

In [ ]:
#We change the reference year by doing the following:
c0.coord('TIME').units='hour since epoch'
c0t = c0.coord('TIME')
#"epoch" = 1st of january of 1970.
print(c0t.units)


In [ ]:
#Now we can safely print the coordinate:
print(c0.coord('TIME'))

In [ ]:
#As the 'time_origin' attribute is still wrong...
print(c0.coord('TIME').attributes['time_origin'])

#We change it by:
c0.coord('TIME').attributes['time_origin'] = '01-JAN-1970 00:00:00'

print(c0.coord('TIME').attributes['time_origin'])

In [ ]:
#We need to extract a 1D cube and for this we create constraints:
print("The original cube:\n {}\n".format(c0))
c1 = c0.extract(iris.Constraint(longitude=45))
print("Extracting a sub-cube with only longitude = 45:\n {}\n".format(c1))
c2 = c1.extract(iris.Constraint(latitude=47))
print("Extracting again a sub-cube with only latitude = 47:\n {}\n".format(c2))

#There's another way of extacting a subset of a cube:
#Another way of slicing a cube:
c3 = c0[:,45,80]
print("Extracting at once by indices:\n {}\n".format(c3))

In [ ]:
#If we print c2.data we get something weird:
if(1): print(c2.data)

#That is because cube thata is a "masked array" object.
print(type(c0.data))

#To show only the valid entries, we use:
print((c0.data.compressed()))

#How many entries are not masked, i.e. valid?
print(c0.data.size)
print(c0.data.compressed().size)

In [ ]:
#That was not a good way of getting a 1D cube. We better take the average.
#Indeed, constraining the cube to just one cell I don't think makes much sense...

#1. We put bound to the cells in order to take an area-average:
if not c0.coord('longitude').has_bounds():
    c0.coord('longitude').guess_bounds()

if not c0.coord('latitude').has_bounds():
    c0.coord('latitude').guess_bounds()

#2. We get the area weights of the cells composing the region:
grid_areas = iris.analysis.cartography.area_weights(c0)

#3. We "collapse" our 2D+Time cube into a 0D+Time by averaging using MEAN aggregator:
c0collapsed = c0.collapsed(['longitude', 'latitude'], iris.analysis.MEAN, weights=grid_areas)

print(c0collapsed)

In [ ]:
#We now want to interpolate between tmin and tmax. 

#We get the bounds:
if not c0.coord('TIME').has_bounds():
    c0.coord('TIME').guess_bounds()
minT = c0.coord('TIME').bounds.min()
maxT = c0.coord('TIME').bounds.max()

print("minT = ",minT)
print("maxT = ",maxT)
print("units = ",c0.coord('TIME').units)

#We set the sample points on the coordinate TIME:
sample_points = [('TIME',np.linspace(minT,maxT))]

#We make a 'Linear' interpolation. The other option is 'Nearest'
c_interp = c0collapsed.interpolate(sample_points, iris.analysis.Linear())

#Notice that the extracted and the interpolated cubes look the same
#except for the number of time points available (12 the original, vs 50 the interpolated)
print("Original collapsed Cube: \n{}\n".format(c0collapsed))
print("Interpolated Cube: \n{}\n".format(c_interp))

In [ ]:
#We now plot the evolution of the temperature:
iplt.plot(c0collapsed,'ro')
iplt.plot(c_interp,'b.')
plt.show()

In [ ]:
#We now do the reverse...

#We get the bounds:
if not c0.coord('longitude').has_bounds():
    c0.coord('longitude').guess_bounds()
minLon = c0.coord('longitude').bounds.min()
maxLon = c0.coord('longitude').bounds.max()

if not c0.coord('latitude').has_bounds():
    c0.coord('latitude').guess_bounds()
minLat = c0.coord('latitude').bounds.min()
maxLat = c0.coord('latitude').bounds.max()

print("minLon = ",minLon)
print("maxLon = ",maxLon)
print("units = ",c0.coord('longitude').units)

print("minLat = ",minLat)
print("maxLat = ",maxLat)
print("units = ",c0.coord('latitude').units)

#We set the sample points on the coordinate TIME:
sample_points = [('longitude',np.linspace(minLon,maxLon)),('latitude',np.linspace(minLat,maxLat))]

#We make a 'Linear' interpolation. The other option is 'Nearest'
c_interpol = c0.interpolate(sample_points, iris.analysis.Linear())
#c_interpol2 = c_interpol.interpolate([('longitude',[45]),('latitude',[47])],iris.analysis.Linear())
#Notice that the extracted and the interpolated cubes look the same
#except for the number of time points available (12 the original, vs 50 the interpolated)
#print("Original collapsed Cube: \n{}\n".format(c0collapsed))
print("Interpolated Cube: \n{}\n".format(c_interpol))
#print("Interpolated Cube2: \n{}\n".format(c_interpol2))

In [ ]:
for coord in c0.coords():
    print(coord.standard_name)
